In [ ]:
import geopandas as geo
import pandas as pd
%config InlineBackend.figure_format ='retina'

In [ ]:
from re import sub
from decimal import Decimal
import matplotlib as plt

In [ ]:
from shapely.geometry import Point, Polygon

In [ ]:
import contextily as cx
import numpy as np

In [ ]:
data=pd.read_csv("~/TX_appleseed_lenders_public/data/ethnicitydata_clean(1).csv")
pdata=pd.read_csv('~/TX_appleseed_lenders_public/data/cleanedpayday(1).csv')

In [ ]:
pgraph = pdata[['Lat','Long']]

In [ ]:
pgeometry = [Point(xy) for xy in zip(pdata['Long'],pdata['Lat'])]

geo_pts_payday = geo.GeoDataFrame(pgraph,geometry=pgeometry,crs="EPSG:4326")

In [ ]:
data['Tract'] = data['Tract'].astype('str')

In [ ]:
data.columns

In [ ]:
data['% Non-White'] = 100*(data['Total:'] - data['Total: White'])/data['Total:']

In [ ]:
interest = 'Total: Black'
graph = data[['Tract',interest]]

In [ ]:
gdf = geo.read_file('~/TX_appleseed_lenders_public/shapefiles/PVS_21_v2_bg_481131.shp')

In [ ]:
gdf = geo.GeoDataFrame(gdf)

In [ ]:
data = graph.join(gdf)

In [ ]:
gdf = geo.GeoDataFrame(data)

In [ ]:
edata=pd.read_csv("~/TX_appleseed_lenders_public/data/evictions_historic_merged.csv")

In [ ]:
edata['GEOID_2020'] = edata['GEOID_2020'].astype('str')
edata['JudgementTotal']=edata['judgment_amt']

In [ ]:
money_df = edata.groupby(by="GEOID_2020").sum()[['JudgementTotal']].reset_index()

In [ ]:
pts = edata[['case_number','lon','lat','JudgementTotal']].copy()

In [ ]:
pts['Size']=pts['JudgementTotal'].apply(lambda x: np.sqrt(x))

In [ ]:
geometry = [Point(xy) for xy in zip(pts['lon'],pts['lat'])]

geo_pts = geo.GeoDataFrame(pts,geometry=geometry,crs="EPSG:4326")

In [ ]:
egdf = geo.read_file('~/TX_appleseed_lenders_public/shapefiles/PVS_21_v2_bg_481131.shp')

In [ ]:
egdf['GEOID']=gdf['BLKGRPID'].apply(lambda x: x[:-1])

In [ ]:
edf = pd.merge(money_df,egdf, left_on='GEOID_2020', right_on='GEOID', how='right')

In [ ]:
egdf = geo.GeoDataFrame(edf[['JudgementTotal','GEOID','geometry']])

In [ ]:
gdf_wm = gdf.to_crs(epsg=3857)
geo_pts_wm = geo_pts.to_crs(epsg=3857)
geo_pts_wm_payday = geo_pts_payday.to_crs(epsg=3857)


base = gdf_wm.replace({0:np.nan}).plot(interest,  
                #scheme='natural_breaks', 
                scheme='quantiles',
                #k=4,
                edgecolor='k',
                cmap='nipy_spectral', legend=True,
                  figsize=(20,20), alpha=0.5)
cx.add_basemap(base,source=cx.providers.OpenStreetMap.Mapnik, zoom=14)
cx.add_basemap(base,source=cx.providers.OpenStreetMap.Mapnik, zoom=12)
geo_pts_wm.plot(ax=base, marker='.', color='purple', markersize=10)
geo_pts_wm_payday.plot(ax=base, marker='*', color='palegreen', markersize=600, alpha=1)

leg = base.get_legend()
leg.set_bbox_to_anchor((1.2, 1.0, 0.0, 0.0))
base.set_axis_off()

base.set_title('Total Black with Lenders and Evictions', fontsize=24)
base.figure.savefig("../results/totalblack.png", dpi=200, bbox_inches="tight")